# Hyperparameter Optimization for Classifier

This notebook is used in Google Colab to run the hyperparameter search for the classifier. The results of each run are uploaded to W&B where the metrics can be aggregated properly.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=e0132d67db355152a3925f1b0367a996c977716084c67122838eac002d321662
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [3]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
import numpy as np
import os
import time
import copy
import random
from sklearn import metrics

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Create the Dataset

The dataset (in folder `plantsdata`) is split into 90/10 train/test splits. During each epoch the metrics are reported to W&B where it is easier to see them all in aggregate over time.

In [5]:
def build_dataset(batch_size):    
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    data_dir = '/content/drive/MyDrive/plantsdata'
    dataset = datasets.ImageFolder(os.path.join(data_dir))

    # 90/10 split
    train_dataset, test_dataset = random_split(dataset, [0.9, 0.1])

    train_dataset.dataset.transform = data_transforms['train']
    test_dataset.dataset.transform = data_transforms['test']

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                               shuffle=True, num_workers=4)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                             shuffle=True, num_workers=4)

    dataloaders = {'train': train_loader, 'test': test_loader}
    dataset_size = len(dataset)
    dataset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}
    class_names = dataset.classes

    return (dataloaders, dataset_sizes)

def build_network():
    network = resnet50(weights=ResNet50_Weights.DEFAULT)
    num_ftrs = network.fc.in_features

    # Add linear layer with number of classes
    network.fc = nn.Linear(num_ftrs, 2)

    return network.to(device)

def build_optimizer(network, optimizer, learning_rate, beta_one, beta_two, eps):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate,
                               betas=(beta_one, beta_two),
                               eps=eps)
    return optimizer

def train_epoch(network, loader, optimizer, criterion, scheduler, dataset_sizes):
    network.train()
    running_loss = 0.0
    running_corrects = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        #loss = F.nll_loss(network(data), target)
        with torch.set_grad_enabled(True):
            outputs = network(data)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, target)
        
        #cumu_loss += loss.item()
        
        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == target.data)

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({'train/batch_loss': loss.item()})

    scheduler.step()

    epoch_loss = running_loss / dataset_sizes['train']
    epoch_acc = running_corrects.double() / dataset_sizes['train']
    
    return (epoch_loss, epoch_acc)

def test(network, loader, optimizer, criterion, dataset_sizes):
    network.eval()
    confusion = torch.empty([0, 1])
    confusion = confusion.to(device)
    running_loss = 0.0
    test_corrects = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        with torch.set_grad_enabled(False):
            outputs = network(data)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, target)

        running_loss += loss.item() * data.size(0)
        test_corrects += torch.sum(preds == target.data)
        
        confusion = torch.cat((confusion, preds[:, None] / target.data[:, None]))

    tp = torch.sum(confusion == 1).item()
    fp = torch.sum(confusion == float('inf')).item()
    tn = torch.sum(torch.isnan(confusion)).item()
    fn = torch.sum(confusion == 0).item()
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f = 2 * ((precision * recall) / (precision + recall))
    
    epoch_loss = running_loss / dataset_sizes['test']
    epoch_acc = test_corrects.double() / dataset_sizes['test']
    
    return (epoch_loss, epoch_acc, precision, recall, f)

In [6]:
def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        (dataloaders, dataset_sizes) = build_dataset(config.batch_size)
        network = build_network()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate, config.beta_one,
                                    config.beta_two, config.eps)
        criterion = nn.CrossEntropyLoss()
        # Decay LR by a factor of 0.1 every 7 epochs
        exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, config.step_size, config.gamma)

        for epoch in range(config.epochs):            
            (epoch_loss, epoch_acc) = train_epoch(network, dataloaders['train'], optimizer,
                                                  criterion, exp_lr_scheduler,
                                                  dataset_sizes)
            wandb.log({"epoch": epoch, 'train/epoch_loss': epoch_loss, 'train/epoch_acc': epoch_acc})
            
            (test_loss, test_acc, test_precision, test_recall, test_f) = test(network, dataloaders['test'],
                                                                              optimizer, criterion,
                                                                              dataset_sizes)
            wandb.log({'test/epoch_loss': test_loss, 'test/epoch_acc': test_acc,
                       'test/precision': test_precision, 'test/recall': test_recall,
                       'test/f1-score': test_f})

## W&B config

These dictionaries specify the parameters which should be optimized as well as how the optimization should be done (`grid`, `random`,…).

In [7]:
sweep_config = {
    'method': 'random'
}

metric = {
    'name': 'test/epoch_acc',
    'goal': 'maximize'   
}

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
    },
}

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'value': 10},
    'batch_size': {
        'values': [4, 8, 16, 32, 64]},
    'learning_rate': {
        'values': [0.1, 0.01, 0.003, 0.001, 0.0003, 0.0001]},
    'step_size': {
        'values': [2, 3, 5, 7]},
    'gamma': {
        'values': [0.1, 0.5]},
    'beta_one': {
        'values': [0.9, 0.99]},
    'beta_two': {
        'values': [0.5, 0.9, 0.99, 0.999]},
    'eps': {
        'values': [1e-08, 0.1, 1]}
})

In [8]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: 9681wnh0
Sweep URL: https://wandb.ai/flower-classification/pytorch-sweeps-demo/sweeps/9681wnh0


In [ ]:
wandb.agent(sweep_id, train, count=60)

wandb: Agent Starting Run: pw52k3j3 with config:
wandb: 	batch_size: 8
wandb: 	beta_one: 0.9
wandb: 	beta_two: 0.99
wandb: 	epochs: 10
wandb: 	eps: 1
wandb: 	gamma: 0.1
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: sgd
wandb: 	step_size: 3
wandb: Currently logged in as: e1527193 (flower-classification). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 236MB/s]


epoch,▁▂▃▃▄▅▆▆▇█
test/epoch_acc,▁▅▆▆██▇▇▇█
test/epoch_loss,█▅▂▂▂▂▁▁▂▂
test/f1-score,▁▆▇▇███▇▇█
test/precision,▁▄▅▅█▇▆▇▅▇
test/recall,▁▇▇▇▆▇█▆█▇
train/batch_loss,▇▆█▅▅▆▆▆▅▃▄▅▄▄▄▅▃█▄█▃▄▂▁▄▃▁▆▅▁▄▆▂▄▂▂▃▄▆▄
train/epoch_acc,▁▆▆▇█▇██▇█
train/epoch_loss,█▅▃▂▁▂▁▁▁▁
epoch,9
test/epoch_acc,0.81111


wandb: Agent Starting Run: ea718wsd with config:
wandb: 	batch_size: 32
wandb: 	beta_one: 0.99
wandb: 	beta_two: 0.99
wandb: 	epochs: 10
wandb: 	eps: 1e-08
wandb: 	gamma: 0.1
wandb: 	learning_rate: 0.1
wandb: 	optimizer: adam
wandb: 	step_size: 2


epoch,▁▃▆█
test/epoch_acc,▁▁▁
test/epoch_loss,█▁▁
test/f1-score,▁▁▁
test/precision,▁▁▁
test/recall,▁▁▁
train/batch_loss,▁▁█▁▁▂▁▁▁▁▂▂▁▁▃▁▁▂▁▁▁▁▂▂▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁
train/epoch_acc,▃▁▇█
train/epoch_loss,█▆▃▁
epoch,3
test/epoch_acc,0.42222


...

wandb: Agent Starting Run: k0hwgfjk with config:
wandb: 	batch_size: 8
wandb: 	beta_one: 0.9
wandb: 	beta_two: 0.9
wandb: 	epochs: 10
wandb: 	eps: 1e-08
wandb: 	gamma: 0.1
wandb: 	learning_rate: 0.003
wandb: 	optimizer: sgd
wandb: 	step_size: 2


epoch,▁▂▃▃▄▅▆▆▇█
test/epoch_acc,▃▁█▆▆▆█▅▅█
test/epoch_loss,▅█▂▂▁▁▁▁▁▁
test/f1-score,▂▁█▇▇▇█▅▅█
test/precision,▇▃█▄▁▁█▁▁█
test/recall,▁▁▆▆██▆▆▆▆
train/batch_loss,▆█▄▅▃▃▇▁▁▁▁▁▁▁▂▁▂▂▁▁▁▁▁▂▁▄▃▁▁▂▂▁▁▃▁▁▁▁▁▁
train/epoch_acc,▁▅██████▇█
train/epoch_loss,█▄▁▁▁▁▁▁▁▁
epoch,9
test/epoch_acc,0.91111


wandb: Agent Starting Run: hb00vz7w with config:
wandb: 	batch_size: 4
wandb: 	beta_one: 0.99
wandb: 	beta_two: 0.9
wandb: 	epochs: 10
wandb: 	eps: 1
wandb: 	gamma: 0.5
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam
wandb: 	step_size: 5


epoch,▁▂▃▃▄▅▆▆▇█
test/epoch_acc,▁▅▆▇▆▇▆▆▆█
test/epoch_loss,█▅▅▃▃▁▃▄▂▁
test/f1-score,▁▄▆▇▆▇▆▆▆█
test/precision,▁█▆▆▂▄▆█▆▄
test/recall,▁▂▅▆▇▇▅▄▅█
train/batch_loss,▅▅▄▆▅▅▂▂▂▃▂▅▃▂▂▁▂▃▂▁█▁▁▂▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁
train/epoch_acc,▁▄▅▇▇▇▇▇██
train/epoch_loss,█▆▄▃▂▂▂▂▁▁
epoch,9
test/epoch_acc,0.9


wandb: Agent Starting Run: 0bg49if5 with config:
wandb: 	batch_size: 8
wandb: 	beta_one: 0.9
wandb: 	beta_two: 0.9
wandb: 	epochs: 10
wandb: 	eps: 1
wandb: 	gamma: 0.1
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: adam
wandb: 	step_size: 2
